In [113]:
library("dplyr")
library("tidyr")

In [114]:
dbName = toString("database.sqlite")

In [115]:
my_db <- src_sqlite(dbName, create = FALSE)

In [116]:
src_tbls(my_db)

[1] "Country"           "League"            "Match"            
[4] "Player"            "Player_Attributes" "Team"             
[7] "Team_Attributes"   "sqlite_sequence"

In [117]:
playerDF = tbl(my_db, sql("SELECT player_api_id,player_name,player_fifa_api_id FROM Player"))

In [118]:
head(playerDF)

Source:   query [?? x 3]
Database: sqlite 3.11.1 [database.sqlite]

  player_api_id        player_name player_fifa_api_id
          <int>              <chr>              <int>
1        505942 Aaron Appindangoye             218353
2        155782    Aaron Cresswell             189615
3        162549        Aaron Doran             186170
4         30572      Aaron Galindo             140161
5         23780       Aaron Hughes              17725
6         27316         Aaron Hunt             158138

In [119]:
playerAttrDF = tbl(my_db, sql("SELECT player_fifa_api_id,player_api_id,overall_rating,gk_diving,gk_handling,gk_kicking,gk_positioning,gk_reflexes FROM Player_Attributes"))

In [120]:
head(playerAttrDF)

Source:   query [?? x 8]
Database: sqlite 3.11.1 [database.sqlite]

  player_fifa_api_id player_api_id overall_rating gk_diving gk_handling
               <int>         <int>          <int>     <int>       <int>
1             218353        505942             67         6          11
2             218353        505942             67         6          11
3             218353        505942             62         6          11
4             218353        505942             61         5          10
5             218353        505942             61         5          10
6             189615        155782             74        14           7
# ... with 3 more variables: gk_kicking <int>, gk_positioning <int>,
#   gk_reflexes <int>

In [121]:
playerAttrDF = mutate(playerAttrDF, gk_rating = (gk_diving + gk_handling + gk_kicking + gk_positioning + gk_reflexes))

In [122]:
playerAttrDF

Source:   query [?? x 9]
Database: sqlite 3.11.1 [database.sqlite]

   player_fifa_api_id player_api_id overall_rating gk_diving gk_handling
                <int>         <int>          <int>     <int>       <int>
1              218353        505942             67         6          11
2              218353        505942             67         6          11
3              218353        505942             62         6          11
4              218353        505942             61         5          10
5              218353        505942             61         5          10
6              189615        155782             74        14           7
7              189615        155782             74        14           7
8              189615        155782             73        14           7
9              189615        155782             73        14           7
10             189615        155782             73        14           7
# ... with more rows, and 4 more variables: gk_kicking <

In [123]:
groupPlayerDF = playerAttrDF %>% 
    group_by(player_api_id) %>%
    summarize(overall_rating = max(overall_rating), gk_rating = max(gk_rating))

In [124]:
groupPlayerDF

Source:   query [?? x 3]
Database: sqlite 3.11.1 [database.sqlite]

   player_api_id overall_rating gk_rating
           <int>          <int>     <int>
1           2625             63       153
2           2752             72        53
3           2768             75       128
4           2770             74       148
5           2790             73       144
6           2796             69       340
7           2802             82       146
8           2805             76       141
9           2857             72       143
10          2862             76       146
# ... with more rows

In [126]:
playerData = left_join(playerDF, groupPlayerDF, by = "player_api_id")

In [127]:
head(playerData)

Source:   query [?? x 5]
Database: sqlite 3.11.1 [database.sqlite]

  player_api_id        player_name player_fifa_api_id overall_rating gk_rating
          <int>              <chr>              <int>          <int>     <int>
1        505942 Aaron Appindangoye             218353             67        43
2        155782    Aaron Cresswell             189615             74       121
3        162549        Aaron Doran             186170             71       125
4         30572      Aaron Galindo             140161             75       111
5         23780       Aaron Hughes              17725             78       129
6         27316         Aaron Hunt             158138             79       132

In [128]:
playerData

Source:   query [?? x 5]
Database: sqlite 3.11.1 [database.sqlite]

   player_api_id        player_name player_fifa_api_id overall_rating gk_rating
           <int>              <chr>              <int>          <int>     <int>
1         505942 Aaron Appindangoye             218353             67        43
2         155782    Aaron Cresswell             189615             74       121
3         162549        Aaron Doran             186170             71       125
4          30572      Aaron Galindo             140161             75       111
5          23780       Aaron Hughes              17725             78       129
6          27316         Aaron Hunt             158138             79       132
7         564793         Aaron Kuhl             221280             61        64
8          30895       Aaron Lennon             152747             84       146
9         528212       Aaron Lennox             206592             48       237
10        101042      Aaron Meijers             1886

In [129]:
topPlayer = tbl_df(playerData) %>% select(player_name,overall_rating) %>% top_n(10) %>% arrange(desc(overall_rating))

Selecting by overall_rating


In [130]:
head(topPlayer, n = 10)

player_name,overall_rating
Lionel Messi,94
Cristiano Ronaldo,93
Gianluigi Buffon,93
Wayne Rooney,93
Gregory Coupet,92
Xavi Hernandez,92
Alessandro Nesta,91
Andres Iniesta,91
Fabio Cannavaro,91
Iker Casillas,91


In [131]:
topGoalKeeper = tbl_df(playerData) %>% select(player_name,gk_rating) %>% top_n(10) %>% arrange(desc(gk_rating))

Selecting by gk_rating


In [132]:
head(topGoalKeeper, n = 10)

player_name,gk_rating
Gianluigi Buffon,449
Gregory Coupet,447
Petr Cech,445
Iker Casillas,442
Manuel Neuer,440
Julio Cesar,438
Sebastian Frey,437
Edwin van der Sar,436
David De Gea,429
Pepe Reina,428


In [201]:
matchTable = tbl(my_db, sql("SELECT league_id,home_team_api_id,away_team_api_id,home_team_goal,away_team_goal,home_player_1,home_player_2,home_player_3,home_player_4,home_player_5,home_player_6,home_player_7,home_player_8,home_player_9,home_player_10,home_player_11,away_player_1,away_player_2,away_player_3,away_player_4,away_player_5,away_player_6,away_player_7,away_player_8,away_player_9,away_player_10,away_player_11 FROM Match"))

In [202]:
matchDF = tbl_df(matchTable)

In [203]:
matchDataDF = gather(matchDF, players, player_api_id, home_player_1 : away_player_11)

In [204]:
head(matchDataDF)

league_id,home_team_api_id,away_team_api_id,home_team_goal,away_team_goal,players,player_api_id
1,9987,9993,1,1,home_player_1,NA
1,10000,9994,0,0,home_player_1,NA
1,9984,8635,0,3,home_player_1,NA
1,9991,9998,5,0,home_player_1,NA
1,7947,9985,1,3,home_player_1,NA
1,8203,8342,1,1,home_player_1,NA


In [178]:
playerMatchesDF = matchDataDF %>% 
    group_by(player_api_id) %>%
    summarize(appearances = n(),league_appearances = n_distinct(league_id))

In [179]:
head(playerMatchesDF)

player_api_id,appearances,league_appearances
2625,90,1
2752,81,1
2768,71,1
2770,16,2
2790,50,1
2796,1,1


In [180]:
playerDatatemp = left_join(tbl_df(playerData), playerMatchesDF, by = "player_api_id")

In [181]:
head(playerDatatemp)

player_api_id,player_name,player_fifa_api_id,overall_rating,gk_rating,appearances,league_appearances
505942,Aaron Appindangoye,218353,67,43,8,1
155782,Aaron Cresswell,189615,74,121,75,1
162549,Aaron Doran,186170,71,125,104,1
30572,Aaron Galindo,140161,75,111,10,1
23780,Aaron Hughes,17725,78,129,162,1
27316,Aaron Hunt,158138,79,132,158,1


In [182]:
topAppearances = select(playerDatatemp,player_name,appearances) %>% top_n(10) %>% arrange(desc(appearances))

Selecting by appearances


In [183]:
head(topAppearances, n = 10)

player_name,appearances
Steve Mandanda,300
Stephane Ruffier,294
Gorka Iraizoz Moreno,286
Samir Handanovic,286
Hugo Lloris,282
Tim Howard,282
Joe Hart,275
Cedric Carrasso,268
Daniel Congre,262
Moussa Sissoko,260


In [184]:
topLeagueAppearances = select(playerDatatemp,player_name,league_appearances) %>% top_n(10) %>% arrange(desc(league_appearances))

Selecting by league_appearances


In [185]:
head(topLeagueAppearances,n = 10)

player_name,league_appearances
Gelson Fernandes,6
Filip Djuricic,5
Ricky van Wolfswinkel,5
Aleksandar Tonev,4
Alfred Finnbogason,4
Almen Abdi,4
Aly Cissokho,4
Angel Di Maria,4
Bojan Krkic,4
Bryan Ruiz,4


In [236]:
teamTable = tbl(my_db, sql("SELECT team_api_id,team_long_name FROM Team"))

In [237]:
teamTable

Source:   query [?? x 2]
Database: sqlite 3.11.1 [database.sqlite]

   team_api_id    team_long_name
         <int>             <chr>
1         9987          KRC Genk
2         9993      Beerschot AC
3        10000  SV Zulte-Waregem
4         9994  Sporting Lokeren
5         9984 KSV Cercle Brugge
6         8635    RSC Anderlecht
7         9991          KAA Gent
8         9998         RAEC Mons
9         7947     FCV Dender EH
10        9985 Standard de Liège
# ... with more rows

In [238]:
teamAttrTable = tbl(my_db, sql("SELECT team_api_id,buildUpPlaySpeed,buildUpPlayDribbling,buildUpPlayPassing,chanceCreationPassing,chanceCreationCrossing,chanceCreationShooting,defencePressure,defenceAggression,defenceTeamWidth FROM Team_Attributes"))

In [239]:
teamAttrTable = mutate(teamAttrTable, chance_creation = (chanceCreationPassing + chanceCreationCrossing + chanceCreationShooting), build_up_play =(buildUpPlaySpeed +buildUpPlayDribbling+buildUpPlayPassing), defense = (defencePressure+defenceAggression+defenceTeamWidth))

In [245]:
groupteamTable = teamAttrTable %>% 
    group_by(team_api_id) %>%
    summarize(chance_creation = max(chance_creation), build_up_play = max(build_up_play), defense = max(defense))

In [246]:
filter(groupteamTable, team_api_id==9930)

Source:   query [?? x 4]
Database: sqlite 3.11.1 [database.sqlite]

  team_api_id chance_creation build_up_play defense
        <int>           <int>         <int>   <int>
1        9930             181           156     150

In [247]:
teamData = left_join(teamTable, groupteamTable, by = "team_api_id")

In [248]:
teamData

Source:   query [?? x 5]
Database: sqlite 3.11.1 [database.sqlite]

   team_api_id    team_long_name chance_creation build_up_play defense
         <int>             <chr>           <int>         <int>   <int>
1         9987          KRC Genk             182           148     205
2         9993      Beerschot AC             185            NA     210
3        10000  SV Zulte-Waregem             153           147     200
4         9994  Sporting Lokeren             154           159     200
5         9984 KSV Cercle Brugge             200           145     195
6         8635    RSC Anderlecht             182           149     190
7         9991          KAA Gent             184           142     166
8         9998         RAEC Mons             168           160     138
9         7947     FCV Dender EH              NA            NA      NA
10        9985 Standard de Liège             182           161     205
# ... with more rows

In [249]:
topBuildUpTeam = tbl_df(teamData) %>% select(team_long_name,build_up_play) %>% top_n(10) %>% arrange(desc(build_up_play))

Selecting by build_up_play


In [250]:
head(topBuildUpTeam, n = 10)

team_long_name,build_up_play
Torino,209
Udinese,205
Fortuna Düsseldorf,198
Podbeskidzie Bielsko-Biała,193
Korona Kielce,192
Carpi,190
Widzew Łódź,190
Widzew Łódź,190
FC Nantes,189
Lazio,188


In [251]:
topChanceCreationTeam = tbl_df(teamData) %>% select(team_long_name,chance_creation) %>% top_n(10) %>% arrange(desc(chance_creation))

Selecting by chance_creation


In [252]:
head(topChanceCreationTeam,n=10)

team_long_name,chance_creation
Lazio,220
SV Werder Bremen,216
Napoli,215
Manchester City,210
Tottenham Hotspur,210
Birmingham City,210
Wolverhampton Wanderers,210
Burnley,210
Swansea City,210
Queens Park Rangers,210


In [253]:
topDefenseTeam = tbl_df(teamData) %>% select(team_long_name,defense) %>% top_n(10) %>% arrange(desc(defense))

Selecting by defense


In [254]:
head(topDefenseTeam, n = 10)

team_long_name,defense
Beerschot AC,210
KVC Westerlo,210
Sporting Charleroi,210
West Bromwich Albion,210
Hull City,210
Birmingham City,210
Wolverhampton Wanderers,210
Blackpool,210
Swansea City,210
Queens Park Rangers,210


In [255]:
teamMatchDF = tbl_df(matchTable)

In [256]:
teamMatchDF

league_id,home_team_api_id,away_team_api_id,home_team_goal,away_team_goal,home_player_1,home_player_2,home_player_3,home_player_4,home_player_5,⋯,away_player_2,away_player_3,away_player_4,away_player_5,away_player_6,away_player_7,away_player_8,away_player_9,away_player_10,away_player_11
1,9987,9993,1,1,NA,NA,NA,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
1,10000,9994,0,0,NA,NA,NA,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
1,9984,8635,0,3,NA,NA,NA,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
1,9991,9998,5,0,NA,NA,NA,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
1,7947,9985,1,3,NA,NA,NA,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
1,8203,8342,1,1,NA,NA,NA,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
1,9999,8571,2,2,NA,NA,NA,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
1,4049,9996,1,2,NA,NA,NA,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
1,10001,9986,1,0,NA,NA,NA,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
1,8342,8571,4,1,NA,NA,NA,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA


In [257]:
teamData

Source:   query [?? x 5]
Database: sqlite 3.11.1 [database.sqlite]

   team_api_id    team_long_name chance_creation build_up_play defense
         <int>             <chr>           <int>         <int>   <int>
1         9987          KRC Genk             182           148     205
2         9993      Beerschot AC             185            NA     210
3        10000  SV Zulte-Waregem             153           147     200
4         9994  Sporting Lokeren             154           159     200
5         9984 KSV Cercle Brugge             200           145     195
6         8635    RSC Anderlecht             182           149     190
7         9991          KAA Gent             184           142     166
8         9998         RAEC Mons             168           160     138
9         7947     FCV Dender EH              NA            NA      NA
10        9985 Standard de Liège             182           161     205
# ... with more rows

In [258]:
teamMatchDataDF = gather(teamMatchDF, team, team_api_id, home_team_api_id,away_team_api_id)

In [259]:
teamMatchDataDF

league_id,home_team_goal,away_team_goal,home_player_1,home_player_2,home_player_3,home_player_4,home_player_5,home_player_6,home_player_7,⋯,away_player_4,away_player_5,away_player_6,away_player_7,away_player_8,away_player_9,away_player_10,away_player_11,team,team_api_id
1,1,1,NA,NA,NA,NA,NA,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,home_team_api_id,9987
1,0,0,NA,NA,NA,NA,NA,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,home_team_api_id,10000
1,0,3,NA,NA,NA,NA,NA,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,home_team_api_id,9984
1,5,0,NA,NA,NA,NA,NA,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,home_team_api_id,9991
1,1,3,NA,NA,NA,NA,NA,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,home_team_api_id,7947
1,1,1,NA,NA,NA,NA,NA,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,home_team_api_id,8203
1,2,2,NA,NA,NA,NA,NA,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,home_team_api_id,9999
1,1,2,NA,NA,NA,NA,NA,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,home_team_api_id,4049
1,1,0,NA,NA,NA,NA,NA,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,home_team_api_id,10001
1,4,1,NA,NA,NA,NA,NA,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,home_team_api_id,8342


In [265]:
homeTeamGoalDF = gather(teamMatchDF, team, team_api_id, home_team_api_id)

In [267]:
homeTeamGoals = homeTeamGoalDF %>% 
    group_by(team_api_id) %>%
    summarize(homeGoals = sum(home_team_goal))

In [268]:
head(homeTeamGoals,n =10)

team_api_id,homeGoals
1601,148
1773,74
1957,176
2033,70
2182,215
2183,96
2186,144
4049,22
4064,13
4087,105


In [269]:
awayTeamGoalDF = gather(teamMatchDF, team, team_api_id, away_team_api_id)

In [270]:
awayTeamGoals = awayTeamGoalDF %>% 
    group_by(team_api_id) %>%
    summarize(awayGoals = sum(away_team_goal))

In [271]:
head(awayTeamGoals,n=10)

team_api_id,awayGoals
1601,132
1773,52
1957,111
2033,68
2182,157
2183,88
2186,91
4049,13
4064,14
4087,75


In [272]:
teamGoalsData = full_join(awayTeamGoals,homeTeamGoals) %>%  mutate(goals = (awayGoals+homeGoals))

Joining, by = "team_api_id"


In [273]:
head(teamGoalsData,n=10)

team_api_id,awayGoals,homeGoals,goals
1601,132,148,280
1773,52,74,126
1957,111,176,287
2033,68,70,138
2182,157,215,372
2183,88,96,184
2186,91,144,235
4049,13,22,35
4064,14,13,27
4087,75,105,180


In [274]:
teamData = left_join(tbl_df(teamData),teamGoalsData,by = "team_api_id")

In [275]:
head(teamData,n=10)

team_api_id,team_long_name,chance_creation,build_up_play,defense,awayGoals,homeGoals,goals
9987,KRC Genk,182,148,205,143,205,348
9993,Beerschot AC,185,NA,210,62,112,174
10000,SV Zulte-Waregem,153,147,200,130,176,306
9994,Sporting Lokeren,154,159,200,130,149,279
9984,KSV Cercle Brugge,200,145,195,94,119,213
8635,RSC Anderlecht,182,149,190,180,247,427
9991,KAA Gent,184,142,166,177,213,390
9998,RAEC Mons,168,160,138,53,76,129
7947,FCV Dender EH,NA,NA,NA,23,21,44
9985,Standard de Liège,182,161,205,142,199,341


In [276]:
topGoalScoringTeams = teamData %>% select(team_long_name,goals) %>% top_n(10) %>% arrange(desc(goals))

Selecting by goals


In [277]:
head(topGoalScoringTeams,n=10)

team_long_name,goals
FC Barcelona,849
Real Madrid CF,843
Celtic,695
FC Bayern Munich,653
PSV,652
Ajax,647
FC Basel,619
Manchester City,606
Chelsea,583
Manchester United,582
